# "BC dates in Python - Part 2 - Astronomy"
> "Several approaches to expressing BC dates in Python using astronomy libraries"

- toc: false
- branch: master
- badges: false
- comments: true
- categories: [datascience, history, python, time]
- image: images/placeholder.png
- hide: false
- search_exclude: false

## Recap: The problem
Python's [datetime module](https://docs.python.org/3/library/datetime.html) has a [MINYEAR](https://docs.python.org/3/library/datetime.html#datetime.MINYEAR) of 1AD, so we can't express BC dates like that. We'll need a different solution.

What are our requirements for a good solution? What functionality are we looking for?
- Expressing BC as well as AD dates
- Create from string and/or numeric parameters
- Print time
- Getters
- Add/subtract time span and getting time deltas
- Get time span delta
- Lightweight objects
- Useable in pandas?

## 2. Scientific libraries
It looks like there aren't a whole lot of historians using Python, but who know who else uses BC dates? Astronomers! There appears to be quite a selection of scientific libraries that deal with dates outside the default Python date range.

### [Astropy](https://www.astropy.org/)
Astropy's [Time](https://docs.astropy.org/en/stable/api/astropy.time.Time.html) module does the job. It supports a wide variety of time scales, formats and precision that are handy in general, plus some highly astronomy specific functionality such as `earth_rotation_angle` and `light_travel_time`. The [documentation](https://docs.astropy.org/en/stable/time/) does a good job explaining how to create and work with `Time`.

This is an incredibly powerful library, and the documentation rocks. The `Time` object supports common operations such as creating, modifying and printing dates and time spans.

One downside is that it has a lot of functionality you likely won't need, so it looks intimidating and confusing at first glance. While it looks like this added functionality would also bloat the object, this isn't a problem in practice. In addition to taking a string as input, `Time` also takes an `ndarray` of strings, and can be interacted with like an array. This means that effectively, you only need to define your time format and such once for the `Time` object, and the individual times can then be accessed by index.

In [1]:
#collapse-hide

#Imports
from astropy.time import Time, TimeDelta

#### Astropy: Creating times BC/AD
One caveat is that we shouldn't use the standard [ISO/ISOT format](https://docs.astropy.org/en/stable/api/astropy.time.TimeISOT.html#astropy.time.TimeISOT), e.g. `2020-01-02T03:04:05`. The ISO format only works for AD dates! Instead, we should use the [FITS format](https://docs.astropy.org/en/stable/api/astropy.time.TimeFITS.html#astropy.time.TimeFITS), e.g. `-00400-01-02T03:04:05`. FITS is an extension to the ISO format that expands the year range to five digits, and supports BC dates. Be aware that for negative dates, you need to pad it out with leading zeroes, or you'll get a parsing exception!

Another limitation is that despite using 5 digits, you can't actually represent dates before 4799 BC (so slightly before the start of the Julian calendar). For a discussion about why this is limitation exists, see [here](https://github.com/astropy/astropy/issues/9231).

In [2]:
#collapse-show

# Example code for creating BC and AD times
ad_date_astro = Time("2020-01-01T01:23:45", format='fits', scale='utc')
print(ad_date_astro)

bc_date_astro = Time("-00400-01-01T01:23:45", format='fits', scale='utc')
print(bc_date_astro)

dates_astro = Time(["-00400-01-01T01:23:45", "2020-01-01T01:23:45"], format='fits', scale='utc')
print(dates_astro)
print(dates_astro[0])

2020-01-01T01:23:45.000
-00400-01-01T01:23:45.000
['-00400-01-01T01:23:45.000' '+02020-01-01T01:23:45.000']
-00400-01-01T01:23:45.000


/usr/local/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


The above warning seems to be benign and from well within Astropy, nothing to do for us here.

Side note: You can get your time data in a different format by using `.isot`, `.unix`. Note that you can print time in formats that wouldn't support parsing it!

In [3]:
#collapse-show

print(ad_date_astro.unix)
print(bc_date_astro.fits)
print(bc_date_astro.isot)

# This line would fail with a ValueError: "Input values did not match the format class isot"
#bc_date_2 = Time(bc_date.isot, format='isot', scale='utc')

1577841825.0
-00400-01-01T01:23:45.000
-400-01-01T01:23:45.000


#### Astropy: Getters
The `Time` class doesn't have an easy way to extract common attributes such as year, month or seconds. It does, however, support `strftime` and provides an implementation based on Python's [`time.strftime`](https://docs.python.org/3/library/time.html#time.strftime). This means we can define our own utility functions to extract the relevant components and parse them into integers for later use.

Sadly, this doesn't work for BC dates since `strftime` internally assumes dates in the `ISO` format for some reason.

In [5]:
#collapse-show

#Getters
def extract_year(t):
    return int(t.strftime('%Y'))

extract_year(ad_date_astro)

# This errors with ValueError: year -400 is out of range
#extract_year(bc_date_astro)

2020

There is always the option of writing our own function to extract time components from the source string based on the FITS representation. We can cheat by detecting if the date is BC and then removing the leading `-`, only to add it back on to the year at the end.

Another limitation is that `datetime.parse` assumes 4 digits for a year with zero-padding, but FITS comes with 5. This means we chop off the leading year digit. This is fine since apparently Astropy can't handle dates before 4799 BC.

In [28]:
#collapse-hide

def get_time_component(time, comp):
    """
    Extract a time component (year, month, day, hour, minute, seconds, microsecond) from Time.

    Parameters
    ----------
    time : Time
        Time to extract from
    comp: string
        time component to extract

    Returns
    -------
    res : int32
        extracted time component
    """
    
    from datetime import datetime
        
    fits_rep = time.fits
    is_bc = False
    if fits_rep.startswith('-'):
        is_bc = True
        fits_rep = fits_rep[1:]
    
    leading_year = None
    if fits_rep.find('-') > 4:
        leading_year = fits_rep[0]
        fits_rep = fits_rep[1:]
    
    dt = datetime.strptime(fits_rep, '%Y-%m-%dT%H:%M:%S.%f')
    if comp == 'y':
        year = dt.year
        if leading_year is not None:
            year += int(leading_year) * 10000
        if is_bc:
            year *= -1
        return year
    elif comp == 'm':
        return dt.month
    elif comp == 'd':
        return dt.day
    elif comp == 'h':
        return dt.hour
    elif comp == 'min':
        return dt.minute
    elif comp == 's':
        return dt.second
    
    return None

In [53]:
print(bc_date_astro.fits)
print(get_time_component(bc_date_astro, 'y'))

print(ad_date_astro.fits)
print(get_time_component(ad_date_astro, 'd'))

far_bc = Time("-04799-01-01T01:23:45", format='fits', scale='utc')
print(get_time_component(far_bc, 'y'))

-00400-01-01T01:23:45.000
-400
2020-01-01T01:23:45.000
1
-4799


### Astropy: Time spans
Astropy comes with a `TimeDelta` class that supports all common time span operations, such as add/subtract and new instance creation. The most common constructors are from seconds or from [`datetime.timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects).

In [ ]:
#collapse-show

delta_astro = ad_date_astro - bc_date_astro
print(delta_astro.datetime)

new_delta_astro = TimeDelta(123456789, format='sec')
print(new_delta_astro.datetime)

new_delta_astro = TimeDelta(timedelta(days=123456), format='datetime')
print(new_delta_astro.datetime)

print((delta_astro - new_delta_astro).datetime)
print(ad_date_astro + new_delta_astro)

### Conclusion
Astropy's time module offers a lot more functionality than we need which makes its objects clunky and its API quite complex. This is offset by having the option to keep an array of datetimes in the `Time` object itself, same as with `TimeDelta`, which lets us manipulate them as we would with `ndarrays`. The downside is that this wouldn't work well with libraries such as Pandas.

### [Skyfield](https://rhodesmill.org/skyfield/time.html)
TODO: Overview

#### Skyfield: Creating times BC/AD


In [ ]:
#collapse-hide

# TODO

#### Skyfield: Getters


In [ ]:
#collapse-hide

# TODO

#### Skyfield: Time spans


In [ ]:
#collapse-hide

# TODO

#### Conclusion
TODO

### [SpiceyPy](https://github.com/AndrewAnnex/SpiceyPy)

This is more of an honourable mention than anything else. SpiceyPy is a wrapper for the C-based [SPICE toolkit](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/info/intrdctn.html), and it shows in it's usability and documentation. For a taste, check out this [easy-to-follow tutorial](https://spiceypy.readthedocs.io/en/main/remote_sensing.html#overview) full of kernel installs and C-style code.

It does come with time utilities, but I couldn't even figure out how to set them up, let alone use them, so maybe steer clear.